In [22]:
# Approach to generate the linear trajectory to every trajectory recived from fiji to then compre
# them by some metrics:

import pandas as pd
import numpy as np
import random

In [25]:
# Creating the sample data frame randomly generated
df_random = pd.DataFrame(columns=['start_x', 'start_y', 'end_x', 'end_y'])

In [26]:
# Generating 10 random positions for start and end points:

for col in df_random.columns:
    df_random[col] = random.sample(range(1,25), 20)
    
df_random

,start_x,start_y,end_x,end_y
0,15,6,21,15
1,2,18,12,4
2,13,19,24,17
3,18,15,17,21
4,14,13,23,19
5,17,22,20,12
6,11,2,5,16
7,12,9,10,6
8,7,14,22,5
9,19,4,15,23
